<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5 color = brown>Predicting Type of Cuisine to serve in a Restaurant</font></h1>

### <font color = brown><i> Business Problem</font></i>
<br>
<font face=calibri size=3>The objective of this report is to help anyone who intends to set up a restaurant or eatery in a neighborhood decide on where and what type of cuisines to serve through analysis of data sourced from foursquare and US census bureau. 
<br/><br/>
“Explore where one should open a restaurant and what type of cuisine to serve through use of data analytics and Machine Learning”.
<br/><br/>
One business strategy coule be to explore cuisine which is not served in the neighbourhood abother could be venture into cuisne mostly served in the area.

</font>


### <font color = brown><i>  Description of Data</font></i>
<br>
<font face=calibri size=3>
For purpose of this report I intend to use data gathered from foursquare about type of cuisine most served at a neighbourhood and US census website about neighbourhood profile through use of API’s. 
<br/><br/>
I will focus on using supervised learning algorithms such as k-Nearest Neighbors and Support Vector Machine and classify data points into categorical type of cuisine.
<br/><br/>
US Census has API’s which provide information about geographies for more than 18000 variables. I plan to use variables pertaining to age, race, income etc. along with data obtained from foursquare regarding number of place, type of cuisine and then predict the type of cuisine can be served when setting a restaurant in a new neighborhood.


#### Getting all the required imports

In [1]:
import requests                           # library to handle requests
import pandas as pd                       # library for data analsysis
import numpy  as np                       # library to handle data in a vectorized manner
from bs4 import BeautifulSoup             # library for parsing the html page
from pandas.io.json import json_normalize # library to tranform JSON file into a pandas dataframe

import geocoder                           # library for getting longitude and latitude of location 

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [1]:
#!pip install census
#!pip install us

from census import Census
from us     import states

ClientAPIKey = 'd16d4d592fbfbf70fe519b47dca283c6ac339d8e'
c = Census(ClientAPIKey)

In [18]:
c.acs5.get('B25034_010E', geo={'for':'state:*', 'in':'county*'})

#https://api.census.gov/data/2017/acs/acs5/subject?get=NAME,S0101_C01_001E&for=county:*&in=state:*&key=d16d4d592fbfbf70fe519b47dca283c6ac339d8e
#c.sf1.get('NAME', geo={'for': 'tract:*','in': 'state:{} county:170'.format(states.AK.fips)})

print(states.MD.fips)

#c = Census("d16d4d592fbfbf70fe519b47dca283c6ac339d8e",year=2017)
#print(c.acs5.get(('NAME', 'B25034_010E'),{'for': 'state:{}'.format(states.MD.fips)}))
#print(c.acs5.get(('NAME', 'B01002_001E'),{'for': 'state:{}'.format(states.MD.fips)}))

#c.acs5.state_county_tract('NAME', [states.AK.fips states.MD.fips], '170', Census.ALL)
#c.acs5.state_county_tract('B01001_004E', {'for': 'state:*'})

CensusException: error: invalid 'in' argument

In [30]:
# Build base URL
HOST = "https://api.census.gov/data"
year = "2010"
dataset = "dec/sf1"
base_url = "/".join([HOST, year, dataset])

# Specify variables and other predicates
get_vars = ["NAME", "P013001", "P037001"]
predicates = {}
predicates["get"] = ",".join(get_vars)
predicates["for"] = "state:*"

# Execute the request, examine text of response object
r = requests.get(base_url, params=predicates)
print(r)

<Response [200]>


In [15]:
#c.sf1.get('NAME', geo={'for': 'tract:*','in': 'state:{} county:170'.format(states.AK.fips)})
c.acs5.state_county( fields='B01001_004E','state_fips: {02} county_fips: {111023}')

SyntaxError: positional argument follows keyword argument (<ipython-input-15-f39f0dfed1dd>, line 2)

In [17]:
c.sf1.get('NAME', geo={'for': 'tract:*',
                       'in': 'state:{} county:170'.format(states.AK.fips)})

print(states.MD.fips)

c = Census("d16d4d592fbfbf70fe519b47dca283c6ac339d8e",year=2017)
print(c.acs5.get(('NAME', 'B25034_010E'),{'for': 'state:{}'.format(states.MD.fips)}))
print(c.acs5.get(('NAME', 'B01002_001E'),{'for': 'state:{}'.format(states.MD.fips)}))

c = Census("d16d4d592fbfbf70fe519b47dca283c6ac339d8e",year=2013)
print(c.acs5.get(('NAME', 'B25034_010E'),{'for': 'state:{}'.format(states.MD.fips)}))
print(c.acs5.get(('NAME', 'B01002_001E'),{'for': 'state:{}'.format(states.MD.fips)}))


24
[{'NAME': 'Maryland', 'B25034_010E': 129556.0, 'state': '24'}]
[{'NAME': 'Maryland', 'B01002_001E': 38.5, 'state': '24'}]
[{'NAME': 'Maryland', 'B25034_010E': 292837.0, 'state': '24'}]
[{'NAME': 'Maryland', 'B01002_001E': 38.0, 'state': '24'}]


In [6]:
acs_data = c.acs5.get(geography = 'zcta', variables = c(total_population = 'B01001_001', 
                                                 median_age = 'B01002_001', 
                                                 male_25_29 = 'B01001_011',
                                                 male_30_34 = 'B01001_012',
                                                 male_35_39 = 'B01001_013',
                                                 male_40_44 = 'B01001_014',
                                                 male_45_49 = 'B01001_015',
                                                 male_50_54 = 'B01001_016',
                                                 male_55_59 = 'B01001_017',
                                                 male_60_61 = 'B01001_018',
                                                 male_62_64 = 'B01001_019',
                                                 male_65_66 = 'B01001_020',
                                                 male_67_69 = 'B01001_021',
                                                 male_70_74 = 'B01001_022',
                                                 male_75_79 = 'B01001_023',
                                                 male_80_84 = 'B01001_024',
                                                 male_85_plus = 'B01001_025',
                                                 female_25_29 = 'B01001_035',
                                                 female_30_34 = 'B01001_036',
                                                 female_35_39 = 'B01001_037',
                                                 female_40_44 = 'B01001_038',
                                                 female_45_49 = 'B01001_039',
                                                 female_50_54 = 'B01001_040',
                                                 female_55_59 = 'B01001_041',
                                                 female_60_61 = 'B01001_042',
                                                 female_62_64 = 'B01001_043',
                                                 female_65_66 = 'B01001_044',
                                                 female_67_69 = 'B01001_045',
                                                 female_70_74 = 'B01001_046',
                                                 female_75_79 = 'B01001_047',
                                                 female_80_84 = 'B01001_048',
                                                 female_85_plus = 'B01001_049',
                                                 leave_630_7 = 'B08011_006',
                                                 leave_7_730 = 'B08011_007',
                                                 leave_730_8 = 'B08011_008',
                                                 leave_8_830 = 'B08011_009',
                                                 leave_830_9 = 'B08011_010',
                                                 walk_to_work = 'B08006_015',
                                                 total_households = 'B11001_002',
                                                 high_school_diploma = 'B15003_017',
                                                 bachelors_degree = 'B15003_022',
                                                 masters_degree = 'B15003_023',
                                                 median_hh_income = 'B19013_001',
                                                 income_hh_50_60 = 'B19001_011',
                                                 income_hh_60_75 = 'B19001_012',
                                                 income_hh_75_100 = 'B19001_013',
                                                 income_hh_100_125 = 'B19001_014',
                                                 income_hh_125_150 = 'B19001_015',
                                                 income_hh_150_200 = 'B19001_016',
                                                 income_hh_200_plus = 'B19001_017',
                                                 male_workers = 'B23022_003',
                                                 female_workers = 'B23022_027',
                                                 renter_occupied = 'B25008_003',
                                                 median_rent = 'B25031_001',
                                                 median_home_value = 'B25077_001'
                                                 ),
              output = 'wide')

TypeError: 'Census' object is not callable

In [8]:
#c.acs5.tables()
x = c.sf1.state_county_tract('NAME', states.AK.fips, '170', Census.ALL)

In [9]:
x

[{'NAME': 'Census Tract 1.01, Matanuska-Susitna Borough, Alaska',
  'county': '170',
  'state': '02',
  'tract': '000101'},
 {'NAME': 'Census Tract 1.02, Matanuska-Susitna Borough, Alaska',
  'county': '170',
  'state': '02',
  'tract': '000102'},
 {'NAME': 'Census Tract 2, Matanuska-Susitna Borough, Alaska',
  'county': '170',
  'state': '02',
  'tract': '000200'},
 {'NAME': 'Census Tract 3, Matanuska-Susitna Borough, Alaska',
  'county': '170',
  'state': '02',
  'tract': '000300'},
 {'NAME': 'Census Tract 4.01, Matanuska-Susitna Borough, Alaska',
  'county': '170',
  'state': '02',
  'tract': '000401'},
 {'NAME': 'Census Tract 4.02, Matanuska-Susitna Borough, Alaska',
  'county': '170',
  'state': '02',
  'tract': '000402'},
 {'NAME': 'Census Tract 5.01, Matanuska-Susitna Borough, Alaska',
  'county': '170',
  'state': '02',
  'tract': '000501'},
 {'NAME': 'Census Tract 5.02, Matanuska-Susitna Borough, Alaska',
  'county': '170',
  'state': '02',
  'tract': '000502'},
 {'NAME': 'Cen